# <font color='#4C5FDA'>**Breast Cancer Detection Based on CNNs Using Thermal Imaging** </font>

Original paper by Juan Pablo Zuluaga, Zeina Al Masry, Khaled Benaggoune, Safa Meraghni & Noureddine Zerhouni: [A CNN-based methodology for breast cancer diagnosis using thermal images](https://www.tandfonline.com/doi/full/10.1080/21681163.2020.1824685)

In [1]:
#@title **Instalar paquetes necesarios**

%%capture
! pip install torchmetrics
! pip install wandb -Uq
# ! pip install onnx

## <font color='#ECA702'>**Clonamos nuestro repo**</font>

Esto con el fin de traer todos los .py para poder entrenar 'localmente' en Colab y registrar las métricas en wandb.

In [2]:
! git clone https://github.com/gpintoruiz/Thermal-Imaging-Breast-Cancer-Detection.git

Cloning into 'Thermal-Imaging-Breast-Cancer-Detection'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 216 (delta 123), reused 113 (delta 45), pack-reused 0
Receiving objects: 100% (216/216), 5.33 MiB | 22.74 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [3]:
%cd Thermal-Imaging-Breast-Cancer-Detection/notebooks

/content/Thermal-Imaging-Breast-Cancer-Detection/notebooks


In [4]:
%ls

0.01-gpr-data-exploration-cv2.ipynb   alexnet.py       train_gkfold.py   validation.py
0.01-gpr-data-exploration-pil.ipynb   make_dataset.py  train_one_run.py  vgg.py
0.04-gpr-colab-experiments.ipynb      preprocess.py    train.py          xception-one-run.yaml
1.00-gpr-xception-from-scratch.ipynb  test.py          utils.py          xception.py


## <font color='#ECA702'>**Configuración inicial para conectarnos con Kaggle**</font>

1. Instalamos kaggle. Para poder usar comandos de Kaggle.

In [5]:
%%capture
! pip install kaggle

Subimos nuestro token de autenticación de Kaggle (si estamos en colab, sino colocarlo en la carpeta del proyecto)

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"majinka10","key":"3d279682cba1b9e369b0c57794d7135e"}'}

1. Creamos los directorios de Kaggle
2. Copiamos nuestro token en .kaggle
3. Con `chmod 600` establecemos los permitos del token en 600, es decir, que solo yo tengo permisos de lectura y escritura sobre el archivo

In [7]:
! mkdir ~/kaggle

In [8]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

## <font color='#ECA702'>**Carga del dataset**</font>

Traemos el dataset [Thermal Images for Breast Cancer Diagnosis DMR-IR](https://www.kaggle.com/datasets/asdeepak/thermal-images-for-breast-cancer-diagnosis-dmrir) desde kaggle.

This dataset is a methodology for breast disease computer-aided diagnosis using dynamic thermography. The thermal images for breast tumors are classified according to DMR-IR standards.

Two types of tumors are classified in this dataset one is benign another is malignant.
- Benign: This type of tumor is usually well-defined and round or oval in shape. (non-cancerous tumor)
- Malignant: This type of tumor is usually poorly defined and irregular with lobules. (cancerous tumor)

In [9]:
%%capture
! kaggle datasets download -d asdeepak/thermal-images-for-breast-cancer-diagnosis-dmrir
! unzip thermal-images-for-breast-cancer-diagnosis-dmrir.zip

Después de descargar los datos. Debemos entender la estructura de las carpetas para poder trabajar con ellas de una mejor manera.
1. La carpeta principal `Imagens e Matrizes da Tese de Thiago Alves Elias da Silva` son todos los datos `data`.
2. La carpeta `12 Novos Casos de Testes` la podemos tomar como nuestro conjunto de prueba (`test`).
3. Mientras que la carpeta `Desenvolvimento da Metodologia` será nuestro conjunto de entrenamiento (`train`).

Luego dentro de nuestras carpetas de `train` y `test` encontramos dos categorías `DOENTES`y `SAUDAтХа├╝VEIS` o SAUDÁVEI. Los primeros son los casos malignos y los segundos benignos.

Dentro de cada una de las carpetas de pacientes saludables y enfermos se encuentran carpetas con números, cada número representa un paciente. Y para cada paciente tendremos dos carpetas más, una para las imágenes **segmentadas** en escala de grises y la otra para la matrix o mapa de calor.

Algo bueno de este dataset es que ya está dividido por pacientes, es decir, no tendremos imagenes del mismo paciente en el conjunto de entrenamiento y testeo. Por lo tanto, vamos a entrenar con N pacientes, y testear con K pacientes, que no son los mismos.

### <font color='#52F17F'>**Partición de los datos**</font>

Este comando nos permite cargar funciones de un .py en el entorno local de Colab. [Fuente](https://stackoverflow.com/questions/47345004/in-googles-colab-notebook-how-do-i-call-a-function-from-a-python-file)

In [10]:
execfile('make_dataset.py')

In [38]:
np.random.seed(2024)

def print_fold_patients(folds: dict, data: pd.DataFrame):
    for fold_name, indices in folds.items():
        train_patients = data.iloc[indices['train']]['patient'].unique()
        val_patients = data.iloc[indices['val']]['patient'].unique()
        test_patients = data.iloc[indices['test']]['patient'].unique()

        print(f"{fold_name}:\n")
        print(f"Train patients: {train_patients}")
        print(f"Validation patients: {val_patients}")
        print(f"Test patients: {test_patients}\n")

# Generar los datos
data = make_dataframe()

# Generar los folds
folds = make_folds(data)

# Imprimir los pacientes por cada fold
print_fold_patients(folds, data)

fold_1:

Train patients: ['48' '69' '14' '63' '65' '62' '66' '61' '64' '45' '44' '43' '11' '46'
 '15' '38' '42' '37' '13' '51' '28' '01' '07' '31' '00' '09' '53' '36'
 '59' '54' '35' '55' '49' '58' '05' '08' '32' '25' '26' '02']
Validation patients: ['16' '40' '41' '39' '04' '24' '29' '56']
Test patients: ['17' '34' '06' '52' '27' '03' '60' '30']

fold_2:

Train patients: ['14' '63' '65' '62' '66' '61' '64' '16' '45' '40' '17' '11' '46' '39'
 '15' '34' '38' '42' '13' '28' '01' '06' '31' '00' '09' '53' '36' '24'
 '54' '52' '55' '58' '05' '08' '32' '03' '60' '25' '30' '02']
Validation patients: ['44' '43' '37' '04' '35' '49' '27' '29']
Test patients: ['48' '69' '41' '51' '07' '59' '56' '26']

fold_3:

Train patients: ['48' '69' '14' '63' '62' '66' '61' '64' '45' '44' '17' '43' '41' '38'
 '37' '13' '51' '28' '06' '04' '07' '31' '00' '09' '53' '36' '24' '59'
 '35' '52' '55' '27' '05' '29' '32' '03' '60' '25' '26' '30']
Validation patients: ['40' '11' '46' '39' '34' '54' '56' '02']
Test pat

## <font color='#ECA702'>**Inicializamos el agende de wandb**</font>

### <font color='#52F17F'>**1. Nos logeamos en nuestra cuenta**</font>

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### <font color='#52F17F'>**2. Hacemos call del agente**</font>


El sweep que estoy probando acá es el [siguiente](https://github.com/gpintoruiz/Thermal-Imaging-Breast-Cancer-Detection/blob/main/notebooks/xception-one-run.yaml). Se pueden cambiar los parámetros a probar como tú quieras de acuerdo con la [documentación](https://docs.wandb.ai/guides/sweeps/define-sweep-configuration) (recomiendo solo cambiar la arquitectura para que las comparaciones entre modelos sean equivalentes).

Si no tienes ni idea qué es un sweep mira el siguiente [tutorial](https://www.youtube.com/watch?v=9zrmUIlScdY&t=1361s&ab_channel=Weights%26Biases).

El comando `--count` sirve para decirle al agente cuántos runs hacer, aplica especialmente cuando el método del sweep es `bayes` o `random`


In [ ]:
! wandb agent aiuis/dip-project/27e1qtt0 --count 30

wandb: Starting wandb agent 🕵️
2024-06-10 19:56:19,681 - wandb.wandb_agent - INFO - Running runs: []
2024-06-10 19:56:20,735 - wandb.wandb_agent - INFO - Agent received command: run
2024-06-10 19:56:20,735 - wandb.wandb_agent - INFO - Agent starting run with config:
	architecture: xception
	augmented: False
	batch_size: 8
	crop: True
	learning_rate: 0.006106816662230798
	normalize: True
	optimizer: sgd
	resize: 150
2024-06-10 19:56:20,737 - wandb.wandb_agent - INFO - About to run command: /usr/bin/env python train_one_run.py --architecture=xception --augmented=False --batch_size=8 --crop=True --learning_rate=0.006106816662230798 --normalize=True --optimizer=sgd --resize=150
2024-06-10 19:56:25,748 - wandb.wandb_agent - INFO - Running runs: ['i159n1q3']
wandb: Currently logged in as: gpintoruiz (aiuis). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: Tracking run with wandb version 0.17.1
wandb: Run data is saved 